1) importation table apprentissage et validation
2) fonction de discretisation
3) discritisation apprentissage et validation (verifier que toute les quali sont de type str)
4) arbre de classifiaction entraine sur la table d'apprentissage
5) Mesure de performance en apprentissage et en validation

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn import metrics
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [62]:
import warnings
warnings.filterwarnings("ignore")

## importation table apprentissage et validation

In [63]:
#import des train_set
train_data=pd.read_table('train_set.csv',sep=",",header=0,index_col=0)
train_data.head()

,assur_caravane,nbmaisons,pouvoir_achat,nbpers_au_foyer,type_client,mt_auto,nb_auto,mt_incendie,nb_incendie,niv_etud_bas,...,marie,autre_relation,assur_sante_prive,assur_sante_public,mt_RC_entreprise,nb_RC_entreprise,mt_camion_livraison,nb_camion_livraison,nb_remorque,mt_remorque
cle,,,,,,,,,,,,,,,,,,,,,
1004,0,1,4,2,9,0,0,0,0,8,...,7,2,1,8,0,0,0,0,0,0
464,0,1,3,2,7,6,1,0,0,3,...,4,2,2,7,0,0,0,0,0,0
542,0,1,3,3,8,6,1,0,0,8,...,9,0,2,7,0,0,0,0,0,0
4009,0,1,4,4,10,0,0,0,0,9,...,5,4,0,9,0,0,0,0,0,0
5158,0,2,2,2,5,6,1,2,1,3,...,2,3,0,9,0,0,0,0,0,0


In [64]:
#import des test_set
test_data=pd.read_table('test_set.csv',sep=",",header=0,index_col=0)
test_data.head()

,assur_caravane,nbmaisons,pouvoir_achat,nbpers_au_foyer,type_client,mt_auto,nb_auto,mt_incendie,nb_incendie,niv_etud_bas,...,marie,autre_relation,assur_sante_prive,assur_sante_public,mt_RC_entreprise,nb_RC_entreprise,mt_camion_livraison,nb_camion_livraison,nb_remorque,mt_remorque
cle,,,,,,,,,,,,,,,,,,,,,
841,0,1,1,1,6,0,0,2,1,5,...,3,5,0,9,0,0,0,0,0,0
3339,0,1,1,4,7,6,1,0,0,9,...,5,4,2,7,0,0,0,0,0,0
2977,0,1,4,3,3,5,1,3,1,7,...,8,1,0,9,0,0,0,0,0,0
5115,0,1,4,3,3,6,1,0,0,3,...,6,2,2,7,0,0,0,0,0,0
528,0,1,1,2,7,6,1,0,0,2,...,9,0,2,7,0,0,0,0,0,0


## Oversampling

In [65]:
train_set_1.index.name = None

In [87]:
liste_des_variables=[
 'pouvoir_achat',
 'niv_etud_bas',
 'niv_etude_haut',
 'revenu1',
 'locataire',
 'marie',
 'revenu3',
 'assur_caravane']
variables_quanti=list(set(liste_des_variables)-set(["assur_caravane"]))

In [88]:
def oversampling(donnees,nom_var):
    data=donnees.copy()
    indexval1=data.index[data[nom_var]=='1']
    indexval0=data.index[data[nom_var]=='0']
    #tirage aléatoire avec remise des valeurs du groupe 1
    newliste=np.random.choice(indexval1,indexval0.shape[0],replace=True).astype('int32')
    newliste1=[]
    for val in newliste:
        newliste1.append(np.min(np.where(data.index==val)))
    data_over=data.take(newliste1)
    data_over=pd.concat([data_over,data.loc[indexval0]],axis=0)
    return(data_over)


train_set_over = oversampling(train_set_1[liste_des_variables],'assur_caravane')#initialisation et choix pour replacement qui permet de spécifier
test_set_over = oversampling(test_set_1[liste_des_variables],'assur_caravane')#initialisation et choix pour replacement qui permet de spécifier



X_train_over=train_set_over[variables_quanti] 
X_test_over=test_set_over[variables_quanti]

y_train_over=train_set_over['assur_caravane']
y_test_over=test_set_over['assur_caravane']

## **KNN** :

In [90]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_data.drop(["assur_caravane"],axis=1))

X_train_cr = scaler.transform(train_data.drop(["assur_caravane"],axis=1))
X_test_cr = scaler.transform(test_data.drop(["assur_caravane"],axis=1))

df_train_cr=pd.concat([pd.DataFrame(X_train_cr,index=train_data.index),train_data['assur_caravane']],axis=1)
df_test_cr=pd.concat([pd.DataFrame(X_test_cr,test_data.index),test_data['assur_caravane']],axis=1)

In [115]:
from sklearn.metrics import confusion_matrix, classification_report

# Bagging

In [116]:
B=1000
n=train_set_over.shape[0]

In [182]:
X1=train_set_over.sample(n,replace=True)
X2=train_set_over.sample(n,replace=True)
X3=train_set_over.sample(n,replace=True)
X4=train_set_over.sample(n,replace=True)
X5=train_set_over.sample(n,replace=True)


X_train1=X1.drop(["assur_caravane"],axis=1)
y_train1=X1['assur_caravane']

X_train2=X2.drop(["assur_caravane"],axis=1)
y_train2=X2['assur_caravane']

X_train3=X3.drop(["assur_caravane"],axis=1)
y_train3=X3['assur_caravane']

X_train4=X3.drop(["assur_caravane"],axis=1)
y_train4=X3['assur_caravane']

X_train5=X3.drop(["assur_caravane"],axis=1)
y_train5=X3['assur_caravane']

In [183]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

y_train=y_train1
y_test=y_test_over

k=40
knn1=KNeighborsClassifier(n_neighbors=k).fit(X_train1,y_train1)
y_predict=knn1.predict(X_train1)
y_predict_test1=knn1.predict(X_test_over)


print("KNN1 pour K=",k)
#calcule de précision-entrainnement
print("calcule de précision-entrainnement : ", knn1.score(X_train1, y_train1))
#calcule de prediction-test
print("calcule de précision-test : ", knn1.score(X_test_over, y_test))

print("\nMatrice de confusion avec LISTE",c," train-validation :")
print(confusion_matrix(y_train,y_predict))

print("\nMatrice de confusion avec LISTE",c," test-validation :")
print(confusion_matrix(y_test,y_predict_test1))

print(classification_report(y_test_over,y_predict_test1))

KNN1 pour K= 40
calcule de précision-entrainnement :  0.730734154126767
calcule de précision-test :  0.5133272058823529

Matrice de confusion avec LISTE 0  train-validation :
[[2951 1474]
 [ 888 3459]]

Matrice de confusion avec LISTE 0  test-validation :
[[905 183]
 [876 212]]
              precision    recall  f1-score   support

           0       0.51      0.83      0.63      1088
           1       0.54      0.19      0.29      1088

    accuracy                           0.51      2176
   macro avg       0.52      0.51      0.46      2176
weighted avg       0.52      0.51      0.46      2176



In [184]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix


y_train=y_train2
y_test=y_test_over

k=40
knn2=KNeighborsClassifier(n_neighbors=k).fit(X_train2,y_train2)
y_predict=knn2.predict(X_train2)
y_predict_test2=knn2.predict(X_test_over)


print("KNN2 pour K=",k)
#calcule de précision-entrainnement
print("calcule de précision-entrainnement : ", knn2.score(X_train2, y_train2))
#calcule de prediction-test
print("calcule de précision-test : ", knn2.score(X_test_over, y_test))

print("\nMatrice de confusion avec LISTE",c," train-validation :")
print(confusion_matrix(y_train,y_predict))

print("\nMatrice de confusion avec LISTE",c," test-validation :")
print(confusion_matrix(y_test,y_predict_test2))

print(classification_report(y_test_over,y_predict_test2))

KNN2 pour K= 40
calcule de précision-entrainnement :  0.719562243502052
calcule de précision-test :  0.5284926470588235

Matrice de confusion avec LISTE 0  train-validation :
[[2717 1678]
 [ 782 3595]]

Matrice de confusion avec LISTE 0  test-validation :
[[928 160]
 [866 222]]
              precision    recall  f1-score   support

           0       0.52      0.85      0.64      1088
           1       0.58      0.20      0.30      1088

    accuracy                           0.53      2176
   macro avg       0.55      0.53      0.47      2176
weighted avg       0.55      0.53      0.47      2176



In [210]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

y_train=y_train3
y_test=y_test_over

k=40
knn3=KNeighborsClassifier(n_neighbors=k).fit(X_train3,y_train3)
y_predict=knn3.predict(X_train3)
y_predict_test3=knn3.predict(X_test_over)


print("KNN3 pour K=",k)
#calcule de précision-entrainnement
print("calcule de précision-entrainnement : ", knn3.score(X_train3, y_train3))
#calcule de prediction-test
print("calcule de précision-test : ", knn3.score(X_test_over, y_test))

print("\nMatrice de confusion avec LISTE",c," train-validation :")
print(confusion_matrix(y_train,y_predict))

print("\nMatrice de confusion avec LISTE",c," test-validation :")
print(confusion_matrix(y_test,y_predict_test3))

print(classification_report(y_test_over,y_predict_test3))

KNN3 pour K= 40
calcule de précision-entrainnement :  0.7181942544459644
calcule de précision-test :  0.44715073529411764

Matrice de confusion avec LISTE 0  train-validation :
[[2577 1754]
 [ 718 3723]]

Matrice de confusion avec LISTE 0  test-validation :
[[753 335]
 [868 220]]
              precision    recall  f1-score   support

           0       0.46      0.69      0.56      1088
           1       0.40      0.20      0.27      1088

    accuracy                           0.45      2176
   macro avg       0.43      0.45      0.41      2176
weighted avg       0.43      0.45      0.41      2176



In [211]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

y_train=y_train4
y_test=y_test_over

k=40
knn4=KNeighborsClassifier(n_neighbors=k).fit(X_train4,y_train4)
y_predict=knn4.predict(X_train4)
y_predict_test4=knn4.predict(X_test_over)


print("KNN4 pour K=",k)
#calcule de précision-entrainnement
print("calcule de précision-entrainnement : ", knn4.score(X_train4, y_train4))
#calcule de prediction-test
print("calcule de précision-test : ", knn4.score(X_test_over, y_test))

print("\nMatrice de confusion avec LISTE",c," train-validation :")
print(confusion_matrix(y_train,y_predict))

print("\nMatrice de confusion avec LISTE",c," test-validation :")
print(confusion_matrix(y_test,y_predict_test4))

print(classification_report(y_test_over,y_predict_test4))

KNN4 pour K= 40
calcule de précision-entrainnement :  0.7181942544459644
calcule de précision-test :  0.44715073529411764

Matrice de confusion avec LISTE 0  train-validation :
[[2577 1754]
 [ 718 3723]]

Matrice de confusion avec LISTE 0  test-validation :
[[753 335]
 [868 220]]
              precision    recall  f1-score   support

           0       0.46      0.69      0.56      1088
           1       0.40      0.20      0.27      1088

    accuracy                           0.45      2176
   macro avg       0.43      0.45      0.41      2176
weighted avg       0.43      0.45      0.41      2176



In [212]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

y_train=y_train5
y_test=y_test_over

k=40
knn5=KNeighborsClassifier(n_neighbors=k).fit(X_train5,y_train5)
y_predict=knn5.predict(X_train5)
y_predict_test5=knn5.predict(X_test_over)


print("KNN5 pour K=",k)
#calcule de précision-entrainnement
print("calcule de précision-entrainnement : ", knn5.score(X_train5, y_train5))
#calcule de prediction-test
print("calcule de précision-test : ", knn5.score(X_test_over, y_test))

print("\nMatrice de confusion avec LISTE",c," train-validation :")
print(confusion_matrix(y_train,y_predict))

print("\nMatrice de confusion avec LISTE",c," test-validation :")
print(confusion_matrix(y_test,y_predict_test5))

print(classification_report(y_test_over,y_predict_test5))

KNN5 pour K= 40
calcule de précision-entrainnement :  0.7181942544459644
calcule de précision-test :  0.44715073529411764

Matrice de confusion avec LISTE 0  train-validation :
[[2577 1754]
 [ 718 3723]]

Matrice de confusion avec LISTE 0  test-validation :
[[753 335]
 [868 220]]
              precision    recall  f1-score   support

           0       0.46      0.69      0.56      1088
           1       0.40      0.20      0.27      1088

    accuracy                           0.45      2176
   macro avg       0.43      0.45      0.41      2176
weighted avg       0.43      0.45      0.41      2176



### Methode du vote (Pour la classification):

In [188]:
def methode_vote(l):
    n=len(l)
    L=[]
    for i in range(len(l[0,:])):
        res=0
        for valeur in l[:,i]:
            res+=int(valeur)
        res=res/n
        L.append(str(int(res)))
    return L

l=np.array([y_predict_test1,y_predict_test2,y_predict_test3,y_predict_test4,y_predict_test5])
y_pred=methode_vote(l)

In [209]:
#calcule de prediction-test
def score(y,y_pred):
    res=0
    for i in range(len(y)):
        if (y[i]==y_pred[i]):
            res+=1
    res=res/len(y)
    return res
            
print("calcule de précision-test : ",score(y_test_over,y_pred))
print(classification_report(y_test_over,y_pred))

calcule de précision-test : 0.6111942543349644
              precision    recall  f1-score   support

           0       0.50      0.90      0.64      1088
           1       0.53      0.12      0.19      1088

    accuracy                           0.51      2176
   macro avg       0.52      0.51      0.42      2176
weighted avg       0.52      0.51      0.42      2176

